# Amostragem e Divisão de Imagens


Este notebook pega uma amostra aleatória das suas imagens JPG e as divide em conjuntos de treino (90%) e validação (10%).

Isso é uma etapa fundamental no pipeline de aprendizado de máquina, garantindo que você tenha dados separados para treinar seu modelo e avaliar seu desempenho em dados que ele nunca viu antes.

## 1. Importar bibliotecas necessárias
Esta célula importa as bibliotecas Python essenciais para o nosso script

In [ ]:
import os
import random 
import shutil
from tqdm import tqdm

## 2. Definir a função de amostragem de imagens
Esta função fará o trabalho principal de:

1. Escanear a pasta de origem em busca de imagens.
2. Amostrar aleatoriamente o número especificado de imagens.
3. Dividir as imagens amostradas em conjuntos de treino (90%) e validação (10%).
4. Copiar as imagens para as pastas respectivas.

In [ ]:
def sample_images(source_folder: str, destination_folder: str, sample_size: int, train_ratio: float = 0.9) -> None:
    print(f"\n --- Iniciando a amostragem e divisão de imagens do seu diretório: '{source_folder}' ---")
    
    # 1. Garantir que as pastas de destino existam
    train_folder = os.path.join(destination_folder, 'train')
    val_folder = os.path.join(destination_folder, 'val')
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(val_folder, exist_ok=True)
    print(f"Pastas de destino ('{train_folder}' e '{val_folder}') estão preparadas.{COLOR_RESET}")

    # 2. Listar todos os arquivos de imagem na pasta de origem e suas subpastas
    all_images = []
    print(f"  {BLUE}Escaneando '{source_folder}' em busca de imagens...{COLOR_RESET}")
    for root, dirs, files in os.walk(source_folder):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                all_images.append(os.path.join(root, file))
    
    # 3. Lidar com o caso onde nenhuma imagem é encontrada
    if not all_images:
        print(f"\n --- AVISO: Ops! Nenhuma imagem JPG/PNG encontrada em '{source_folder}' ou suas subpastas. ---")
        print(f"Por favor, verifique o caminho especificado e a existência das imagens.")
        return

    print(f"  {BOLD}Total de {len(all_images)} imagens encontradas para processamento.")

    # 4. Verificar se o tamanho da amostra é válido e ajustar se necessário
    if sample_size <= 0:
        print(f"\n--- ERRO: O tamanho da amostra deve ser um número positivo. A operação foi abortada. ---")
        return
    if sample_size > len(all_images):
        print(f"\n--- AVISO: O tamanho da amostra solicitado ({sample_size}) é maior do que o número total de imagens disponíveis ({len(all_images)}). ---")
        sample_size = len(all_images)
        print(f"Amostra ajustada para incluir todas as {sample_size} imagens disponíveis.")

    # 5. Amostrar imagens aleatoriamente
    sampled_images = random.sample(all_images, sample_size)
    print(f"Foram selecionadas aleatoriamente {len(sampled_images)} imagens para a amostra final.")

    # 6. Calcular a divisão (treino/validação)
    train_size = int(train_ratio * sample_size)
    val_size = sample_size - train_size # O restante vai para validação

    # 7. Embaralhar a amostra antes de dividir para garantir aleatoriedade na divisão
    random.shuffle(sampled_images)
    train_images = sampled_images[:train_size]
    val_images = sampled_images[train_size:]

    print(f"Divisão da amostra: Treino = {len(train_images)} imagens, Validação = {len(val_images)} imagens.")

    # 8. Copiar as imagens amostradas para as pastas de destino
    print(f"\n--- Copiando imagens para o conjunto de TREINO em '{train_folder}' ---")
    for img_path in tqdm(train_images):
        dst_path = os.path.join(train_folder, os.path.basename(img_path))
        try:
            shutil.copy2(img_path, dst_path)
        except Exception as e:
            print(f"ERRO AO COPIAR: Não foi possível copiar '{os.path.basename(img_path)}'.")
            print(f"Detalhes: {e}. Verifique as permissões de arquivo ou o caminho de origem/destino.")

    print(f"\n --- Copiando imagens para o conjunto de VALIDAÇÃO em '{val_folder}' ---")
    for img_path in tqdm(val_images):
        dst_path = os.path.join(val_folder, os.path.basename(img_path))
        try:
            shutil.copy2(img_path, dst_path)
        except Exception as e:
            print(f"ERRO AO COPIAR: Não foi possível copiar '{os.path.basename(img_path)}'.")
            print(f"Detalhes: {e}. Verifique as permissões de arquivo ou o caminho de origem/destino.")

## 3. Definir parâmetros de entrada

Esta célula define as variáveis de entrada para o script:

In [ ]:
source_folder = 'JPG_Images'  # Onde suas imagens JPG convertidas estão localizadas (do Notebook 2)
destination_folder = 'sampled_datasets' # Onde os conjuntos de treino e validação serão criados
sample_size = 1000 # O número total de imagens que você quer amostrar
train_ratio = 0.9  # 90% para treino, 10% para validação

## 4. Executar a função de amostragem

Essa célula executará o processo de amostragem com base em sua entrada:

In [ ]:
print(f"\nParâmetros definidos para esta execução:")
print(f"  Pasta de Origem: {source_folder}")
print(f"  Pasta de Destino: {destination_folder}")
print(f"  Tamanho da Amostra: {sample_size} imagens")
print(f"  Proporção de Treino: {train_ratio*100}%")

try:
    sample_images(source_folder, destination_folder, sample_size, train_ratio)
    print(f"\n --- Amostragem de imagens concluída com sucesso! ---")
    print(f"Seus conjuntos de dados estão prontos. Verifique-os em:")
    print(f"Treino: {os.path.abspath(os.path.join(destination_folder, 'train'))}")
    print(f"Validação: {os.path.abspath(os.path.join(destination_folder, 'val'))}")
except Exception as e:
    print(f"\n --- ERRO CRÍTICO: Ocorreu um problema durante a amostragem de imagens! ---")
    print(f"Detalhes do erro: {str(e)}")
    print(f"Por favor, verifique se os caminhos das pastas estão corretos e se você tem as permissões necessárias para leitura e escrita.")